In [3]:
type automate = int * (int * int) array * bool array ;;

let a1 = 2, [|(1, 1) ; (0, 0)|], [|false; true|] ;;
let a2 = 2, [|0, 1 ; 1, 0|], [|false; true|]  ;;
let a3 = 3, [|(1, 2); (1, 2); (2, 1)|], [|false; false; true|]
let a4 = 4, [|(1, 2) ; (0, 3); (3, 0); (2, 1)|], [|false ; false ; true ; true|]
let a5 = 3, [|(0, 1); (2, 0) ; (1, 0)|], [|false; false ; true|]
let a6 = 6, [|(0, 2) ; (0, 5); (3, 1); (2, 4); (5, 4); (3, 0)|], [|true ; true ; false ; false ; false ; false|]

type automate = int * (int * int) array * bool array


val a1 : int * (int * int) array * bool array =
  (2, [|(1, 1); (0, 0)|], [|false; true|])


val a2 : int * (int * int) array * bool array =
  (2, [|(0, 1); (1, 0)|], [|false; true|])


val a3 : int * (int * int) array * bool array =
  (3, [|(1, 2); (1, 2); (2, 1)|], [|false; false; true|])


val a4 : int * (int * int) array * bool array =
  (4, [|(1, 2); (0, 3); (3, 0); (2, 1)|], [|false; false; true; true|])


val a5 : int * (int * int) array * bool array =
  (3, [|(0, 1); (2, 0); (1, 0)|], [|false; false; true|])


val a6 : int * (int * int) array * bool array =
  (6, [|(0, 2); (0, 5); (3, 1); (2, 4); (5, 4); (3, 0)|],
   [|true; true; false; false; false; false|])


In [4]:
let numero n a =
  let t = Array.make n (-1) in
  let rec aux index = function (*parcours la liste*)
    |[] -> ()
    |h::q -> t.(h) <- index ; aux (index + 1) q in
  aux 0 a ; t ;;

val numero : int -> int list -> int array = <fun>


In [5]:
let etats_accessibles aut = 
  let n, delta, f = aut in 
  let visited = Array.make n false in
  let parcours = ref [] in
  let rec aux etat =
      if not visited.(etat) then 
          begin
          visited.(etat) <- true ;
          parcours := etat :: !parcours ;
          let succ_a, succ_b = delta.(etat) in
          aux succ_a ;
          aux succ_b
          end 
  in aux 0 ;
  List.rev !parcours ;;

val etats_accessibles : int * (int * int) array * 'a -> int list = <fun>


In [6]:
let partie_accessible aut = 
    let n, delta, f = aut in
    let new_etats = etats_accessibles aut in 
    let apparition = numero n new_etats in
    let new_n = List.length new_etats in
    let new_delta = Array.make new_n (0,0) in
    let new_f = Array.make new_n false in
    let rec aux = function
        |[] -> (new_n, new_delta, new_f)
        |h::t -> let s = apparition.(h) in
                new_f.(s) <- f.(h);
                let succ_a, succ_b = delta.(h) in
                new_delta.(s) <- apparition.(succ_a) ,apparition.(succ_b);
                aux t in
    aux new_etats ;;


val partie_accessible :
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array = <fun>


In [7]:
let existe_morphismes aut1 aut2 =
    let n1, delta1, f1 = aut1 in 
    let n2, delta2, f2 = aut2 in
    let def = ref true in (*variable indiquant si un morphisme existe ou non*)
    let visited = Array.make n1 false in
    let morphisme = Array.make n1 (-1) in 
    let construire etat1 etat2 = (*fonction qui construit le morphisme*)
        if morphisme.(etat1) <> 1 (*si phi(q) ne s'est pas encore vu etre associe une image, on la definit*)
            then begin
                if ( f1.(etat1) = f2.(etat2) ) || ( (not f1.(etat1)) = (not f2.(etat2)) ) (*test de la condition (4)*)
                then morphisme.(etat1) <- etat2 (*si (4) est respectee, alors on peut definir phi(q)*)
                else def := false (*sinon il n'existe pas de morphisme*)
            end
        else if morphisme.(etat1) <> etat2 (*si phi(q) est deja defini, mais que la condition (3) n'est pas respectee...*)
             then def := false (*... alors il n'existe pas de morphisme*)
    in
    let rec aux etat1 = (*fonction qui parcours l'automate afin de constuire le morphisme*)
        if not visited.(etat1) (*on verifie que le sommet n'a pas deja ete visite*)
        then begin
            let etat2 = morphisme.(etat1) in
            let succ1_a, succ1_b = delta1.(etat1) in (*on construit le morphisme recursivement, en partant du sommet que l'on visite*)
            let succ2_a, succ2_b = delta2.(etat2) in
            visited.(etat1) <- true ;
            construire succ1_a succ2_a ; (*on definit les images de phi pour sigma = a*)
            construire succ1_b succ2_b ; (*on definit les images de phi pour sigma = b*)
            if !def then (aux succ1_a ; aux succ1_b) (*si le morphisme existe (i.e les etapes de construction se sont achevees), alors on continue jusqu'a ce que tous les sommets soient visites*)
            end 
    in
    morphisme.(0) <- 0 ; (*initialisation de la constuction avec la condition (2)*)
    aux 0 ; (*début du parcours*)
    !def , morphisme ;;      

val existe_morphismes :
  int * (int * int) array * bool array ->
  'a * (int * int) array * bool array -> bool * int array = <fun>


In [8]:
let produit aut1 aut2 = 
    let n1, delta1, f1 = aut1 in 
    let n2, delta2, f2 = aut2 in 
    let n = n1 * n2 in 
    let f = Array.make n false in
    let delta = Array.make n (0, 0) in
    let prod_etats n x1 x2 = match x1, x2 with
|(a, b), (c, d) -> n*a + c, n*b + d in
    for i = 0 to n1 - 1 do
        for j = 0 to n2 - 1 do
            begin
            if f1.(i) && f2.(j) then
            f.(n2 * i + j) <- true ;
            delta.(n2 * i + j) <- prod_etats n2 delta1.(i) delta2.(j) ;
            end
        done;
    done;
    partie_accessible (n, delta, f) ;

val produit :
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array = <fun>


In [9]:
let maxi_positif arr = (*renvoie l'element maximum d'un tableau d'entiers positifs (et -1 si la liste est vide)*)
  if Array.length arr = 0 then -1 else
  let value = ref arr.(0) in 
  for i=1 to (Array.length arr) - 1 do
      value := max arr.(i) !value
  done; 
  !value ;; 

let renomme arr =
    let size = Array.length arr in 
    let l = maxi_positif arr + 1 in (*entier l positif permettant de creer le dictionnaire (cf ci dessous)*)
    let index = Array.make l (-1) in (*semblant de dictionnaire qui a une valeur n associe une cle k (son indice dans le tableau renomme)*)
    let res = Array.make size 0 in (*tableau renomme*)
    let compteur = ref 0 in (*compteur permettant de garder en memoire la derniere cle associe a une valeur*)
    for i = 0 to size - 1 do
        begin
        if index.(arr.(i)) = -1 (*verifie que l'on a pas deja associe a la valeur n une cle*)
            then begin index.(arr.(i)) <- !compteur ;  (*on attribut a la valeur n une cle*)
                       incr compteur (*on incremente le compteur*)
                 end ;
        res.(i) <- index.(arr.(i)) (*on renomme le tableau*)
        end
    done;
    res ;;

val maxi_positif : int array -> int = <fun>


val renomme : int array -> int array = <fun>


In [10]:
let construction_graphe morph = (*fonction finalement inutile*)
  let qA = Array.length morph in 
  let qB = maxi_positif morph + 1 in
  let voisins = Array.make qB [] in
  for i=0 to qA - 1 do
    voisins.(morph.(i)) <- i :: voisins.(morph.(i))
  done;
  let graphe = Array.make qA [] in
  let rec aux i list = 
      match list with
      |[] -> []
      |h::t -> if h = i then aux i t else h::aux i t
  in 
  for i=0 to qA - 1 do
    graphe.(i) <- aux i voisins.(morph.(i))
  done ;
  graphe ;;

val construction_graphe : int array -> int list array = <fun>


In [11]:
let relation morph1 morph2 =
  let q_depart = Array.length morph1 in (*cardinal de l'automate de depart*)
  let q_arrive1 = maxi_positif morph1 + 1 in (*cardinal de l'automate image de phi*)
  let q_arrive2 = maxi_positif morph2 + 1 in (*cardinal de l'automate image de psi*)
  let voisins1 = Array.make q_arrive1 [] in (*voisins1.(q) = liste des antecedents de q par phi*)
  let voisins2 = Array.make q_arrive2 [] in (*voisins2.(q) = liste des antecedents de q par psi*)
  for i = 0 to (q_depart - 1) do (*on remplit les voisins*)
    voisins1.(morph1.(i)) <- i :: voisins1.(morph1.(i)) ;
    voisins2.(morph2.(i)) <- i :: voisins2.(morph2.(i))
  done;
  let morph_res = Array.make q_depart (-1) in (*initialisation de eta*)
  let rec aux e arr = function (*fonction auxiliaire qui pour tout i dans la liste va attribuer a arr.(i) la valeur e*)
    |[] -> ()
    |h::t -> morph_res.(h) <- e ; aux e arr t in
  for i = 0 to (q_depart - 1) do (*on remplit eta*)
    if morph_res.(i) = -1 then (*verifie si l'etat i ne s'est pas deja vu etre attribue une classe d'equivalence ...*)
      begin (*... si non, alors on initialise la classe d'equivalence (en lui donnant un numero quelconque (MAIS QUI N'A PAS ENCORE ETE DONNE))*)
        aux i morph_res voisins1.(morph1.(i)) ; (*on attribut cette classe d'equivalence aux voisins de q par phi*)
        aux i morph_res voisins2.(morph2.(i)) ; (*on attribut cette classe d'equivalence aux voisins de q par psi*)
      end
    else begin (*... si on a deja attribue une classe d'equivalence a cet etat, alors on l'attribut aux voisins de cet etat*)
      aux morph_res.(i) morph_res voisins1.(morph1.(i)) ;
      aux morph_res.(i) morph_res voisins2.(morph2.(i)) ;
    end
    done;
    renomme morph_res ;; (*on renomme les classes d'equivalences (notamment afin d'avoir morph_res.(0) = 0)*)

val relation : int array -> int array -> int array = <fun>


In [12]:
let table_de_predecesseurs aut =
    let n, delta, f  = aut in 
    let tab = Array.make_matrix n n false in (*on initalise le tableau*)
    let rec aux p q = (*fonction auxiliaire qui visite tous les (p, q) pour un couple (p0, q0) donnee*)
      if not tab.(p).(q)  (*s'assure de ne visiter les sommets qu'une seule fois*)
        then begin
          tab.(p).(q) <- true ; 
          let succP_a, succP_b = delta.(p) in
          let succQ_a, succQ_b = delta.(q) in
          aux succP_a succQ_a ; (*début du parcours*)
          aux succP_b succQ_b ; (*suite du parcours (en profondeur)*)
        end
    in 
    for p = 0 to n - 1 do 
      for q = 0 to n - 1 do
        if (f.(p) && (not f.(q))) || ((not f.(p)) && f.(q)) (*pour tous les couples (p, q), on regarde si il respecte les conditions de l'enonce ...*)
          then aux p q (*... si oui, alors on le visite*)
      done;
    done;
    tab ;;

val table_de_predecesseurs :
  int * (int * int) array * bool array -> bool array array = <fun>


In [61]:
let table_de_predecesseurs_bis aut = 
  let n, delta, f = aut in
  let tab = Array.make_matrix n n false in
  let transi = Array.make_matrix n n [] in (*graphe P'*)
  for p = 0 to n - 1 do (*construction de P'*)
      for q = 0 to n - 1 do
          let succP_a, succP_b = delta.(p) in
          let succQ_a, succQ_b = delta.(q) in
          transi.(succP_a).(succQ_a) <- (p, q) :: transi.(succP_a).(succQ_a);
          transi.(succP_b).(succQ_b) <- (p, q) :: transi.(succP_b).(succQ_b);
      done;
  done;
  let rec aux p q = (*fonction auxiliaire qui parcours P' pour mettre a jour les relations "etoiles" entre les etats*)
    if not tab.(p).(q)
    then begin
    tab.(p).(q) <- true ; 
    List.iter (fun (i,j) -> aux i j) transi.(p).(q) ;
    end
  in
  for p = 0 to n - 1 do 
      for q = 0 to n - 1 do
          if (f.(p) && (not f.(q))) || ((not f.(p)) && f.(q))
          then aux p q
      done;
  done;
  tab ;;

val table_de_predecesseurs_bis :
  int * (int * int) array * bool array -> bool array array = <fun>


In [62]:
table_des_predecesseurs_bis a6

- : bool array array =
[|[|false; false; true; true; true; true|];
  [|false; false; true; true; true; true|];
  [|true; true; false; true; true; false|];
  [|true; true; true; false; false; true|];
  [|true; true; true; false; false; true|];
  [|true; true; false; true; true; false|]|]


In [63]:
let etoile aut = 
    let n, delta, f = aut in
    let pred = table_de_predecesseurs_bis aut in
    let classes_equiv = Array.make n (-1) in
    for p = 0 to n - 1 do
      if classes_equiv.(p) = -1
      then begin 
          classes_equiv.(p) <- p ;
          for q = 0 to n - 1 do 
              if not pred.(p).(q) 
              then classes_equiv.(q) <- p
          done;
          end
    done;
    let classes_equiv = renomme classes_equiv in
    let nb_classes = maxi_positif classes_equiv + 1 in
    let etats_classe = Array.make nb_classes (-1) in
    Array.iteri (fun i elem -> etats_classe.(elem) <- i) classes_equiv ;
    classes_equiv, etats_classe ;;

val etoile : int * (int * int) array * bool array -> int array * int array =
  <fun>


In [64]:
etoile a6

- : int array * int array = ([|0; 0; 1; 2; 2; 1|], [|1; 5; 4|])


In [65]:
let reduit aut = 
  let n, delta, f = aut in 
  let classes_equiv, etats_classe = etoile aut in
  let new_n = Array.length etats_classe in 
  let new_delta = Array.make new_n (-1, -1) in 
  let new_f = Array.make new_n false in 
  for p = 0 to new_n - 1 do
    let succ_a, succ_b = delta.(etats_classe.(p)) in 
    new_delta.(p) <- classes_equiv.(succ_a), classes_equiv.(succ_b) ;
    new_f.(p) <- f.(etats_classe.(p))
  done ;
  (new_n, new_delta, new_f) ;;

val reduit :
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array = <fun>


In [66]:
reduit a6

- : int * (int * int) array * bool array =
(3, [|(0, 1); (2, 0); (1, 2)|], [|true; false; false|])
